In [1]:
import pandas as pd
import razdel
from razdel import tokenize
import string
import re
from transformers import AutoTokenizer, BertForTokenClassification, pipeline
import numpy as np
from datasets import load_dataset
import transformers
import torch
from torch import nn

/home/qklent/programming/machine_learning/alfa_bank_receipts/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
transformers.set_seed(1337)

In [3]:
path_to_train_raw_data = "../data/train_supervised_dataset.csv"

In [4]:
df = pd.read_csv(path_to_train_raw_data)
df.head()

,id,name,good,brand
0,0,Petmax Бантик леопард с красн розой 2шт,бантик,petmax
1,1,87191 Бусы для елки шарики_87191,бусы,NaN
2,2,Футболка Piazza Italia WR011446881,футболка,piazza italia
3,3,7) YI572-03X-ONE ЗАКОЛКА ДЛЯ ВОЛОС ДЛЯ ДЕВОЧКИ,заколка,NaN
4,4,Одежда (вес) 1500,одежда,NaN


In [5]:
df = df.fillna("")

In [6]:
#тут можно еще сделать ru_to_eng(дада и такие странные примеры есть)
def eng_to_ru(word: str) -> str:
    if len(word) <= 1:
        return word
    a = ord('а')
    ru_alphabet = ''.join([chr(i) for i in range(a,a+33)])
    eng_alphabet = string.ascii_lowercase[:26]
    change = {
        "a": "а",
        "e": "е",
        "o": "о",
        "k": "к",
        "3": "з",
        "p": "р",
        "c": "с",
        "m": "м",
        "x": "х",
        "t": "т",
        "y": "у",
        "z": "з",
    }

    new_word = ""
    for i in range(len(word)):
        curr = word[i]
        if curr in eng_alphabet:
            if i - 1 < 0:
                prev_letter = "j"
            else:
                prev_letter = word[i - 1]

            if i + 1 >= len(word):
                next_letter = "j"
            else:
                next_letter = word[i + 1]

            if next_letter in ru_alphabet or prev_letter in ru_alphabet:
                curr = change.get(curr, curr)
        new_word += curr

    return new_word
# to add: seperate by string.punctuations (add space between words)
#(probably i can just use some tokenizer and then make string of tokenizer output)


In [7]:
def only_ones(word: str) -> str:
    if word.isdigit() or re.match(r'^-?\d+(?:\.\d+)$', word):
        return "1"
    else:
        new_word = ""
        i = 0
        while i < len(word):
            curr = word[i]
            while i < len(word) and word[i].isdigit():
                curr = "1"
                i += 1
            new_word += curr
            if curr != "1":
                i += 1

    return new_word


In [8]:
def remove_punct(word: str) -> str:
    word = re.sub(r'[^\w\s]+', ' ', word)
    word = re.sub(r'\s+', ' ', word)
    return word.strip()


def preprocess_string(word: str) -> str:
    new_word = ""


    word = remove_punct(word)

    if word != "товара нет":
        new_word = eng_to_ru(only_ones(word).lower())

    return new_word

In [9]:
for i in range(1):
    print()
    q = tokenize(df.name[i])
    for x in q:
        print(x)



Substring(0, 6, 'Petmax')
Substring(7, 13, 'Бантик')
Substring(14, 21, 'леопард')
Substring(22, 23, 'с')
Substring(24, 29, 'красн')
Substring(30, 35, 'розой')
Substring(36, 37, '2')
Substring(37, 39, 'шт')


In [10]:
names = df.name.apply(preprocess_string)
names

0                  petmax бантик леопард с красн розой 1шт
1                                 1 бусы для елки шарики_1
2                               футболка piazza italia wr1
3               1 yi1 1x one заколка для волос для девочки
4                                             одежда вес 1
                               ...                        
24995                                  вода саирме с г 1мл
24996                             моя семя 1 1л и ассортим
24997             рулет бисквитн яшкино клубничный со слив
24998    1 почвогрунт цветочное счастье фаско декоратив...
24999           семечки жар 1г крутой окер с солью гринвич
Name: name, Length: 25000, dtype: object

In [11]:
def tokenize_names(names: pd.Series) -> pd.Series:
    return [list(map(lambda x: x.text, tokenize(x))) for x in names]

names = tokenize_names(names)
#delete solo symbols
for i in range(len(names)):
    new_name = []
    for j in range(len(names[i])):
        if len(names[i][j]) > 1:
            new_name.append(names[i][j])
    names[i] = new_name
names[:2]

[['petmax', 'бантик', 'леопард', 'красн', 'розой', 'шт'],
 ['бусы', 'для', 'елки', 'шарики_1']]

In [12]:
brands = df.brand.apply(preprocess_string)
goods = df.good.apply(preprocess_string)

In [13]:
brands[:10], goods[:10]

(0           petmax
 1                 
 2    piazza italia
 3                 
 4                 
 5        русалочка
 6                 
 7       очаковский
 8             cosy
 9              ekf
 Name: brand, dtype: object,
 0        бантик
 1          бусы
 2      футболка
 3       заколка
 4        одежда
 5         губка
 6          обои
 7          квас
 8         маска
 9    наконечник
 Name: good, dtype: object)

In [14]:
list(tokenize(brands[2]))

[Substring(0, 6, 'piazza'), Substring(7, 13, 'italia')]

In [15]:
#tagging every tokken if it has label or nah
# нужно текенизировать brand and goods
label_names = ["O", "B-BRAND", "I-BRAND", "B-GOOD", "I-GOOD"] # "O" stand for out of context or smth like this
skipped = 0
def label_name(name: list, brand: str, good: str) -> list:
    """
    gives label to every word in 1 receipt
    """
    labels = ["O" for _ in range(len(name))]
    brand = [x.text for x in tokenize(brand)]
    good = [x.text for x in tokenize(good)]
    numeric_labels = [0 for _ in range(len(name))]
    global skipped

    if brand:
        try:
            idx = name.index(brand[0])
            labels[idx] = label_names[1]
            numeric_labels[idx] = 1

            for x in brand[1:]:
                idx = name.index(x)
                labels[idx] = label_names[2]
                numeric_labels[idx] = 2
        except ValueError:
            skipped += 1
            return None

    if good:
        try:
            idx = name.index(good[0])
            labels[idx] = label_names[3]
            numeric_labels[idx] = 3

            for x in good[1:]:
                idx = name.index(x)
                labels[idx] = label_names[4]
                numeric_labels[idx] = 4
        except ValueError:
            skipped += 1
            return None
    return name, labels, numeric_labels



In [16]:
i = 0
names[i]

['petmax', 'бантик', 'леопард', 'красн', 'розой', 'шт']

In [17]:
label_name(names[i], brands[i], goods[i])

(['petmax', 'бантик', 'леопард', 'красн', 'розой', 'шт'],
 ['B-BRAND', 'B-GOOD', 'O', 'O', 'O', 'O'],
 [1, 3, 0, 0, 0, 0])

In [18]:
name_and_labels = []
skipped = 0
real_brands = []
real_goods = []
for n, b, g in zip(names, brands, goods):
    #print (n, b)
    curr = label_name(n, b, g)
    if curr is not None:
        real_brands.append(b)
        real_goods.append(g)
        name_and_labels.append(curr)

In [19]:
print(f"{skipped} of {len(df)} examples were skipped")

3734 of 25000 examples were skipped


In [20]:
name_and_labels[:3]

[(['petmax', 'бантик', 'леопард', 'красн', 'розой', 'шт'],
  ['B-BRAND', 'B-GOOD', 'O', 'O', 'O', 'O'],
  [1, 3, 0, 0, 0, 0]),
 (['бусы', 'для', 'елки', 'шарики_1'],
  ['B-GOOD', 'O', 'O', 'O'],
  [3, 0, 0, 0]),
 (['футболка', 'piazza', 'italia', 'wr'],
  ['B-GOOD', 'B-BRAND', 'I-BRAND', 'O'],
  [3, 1, 2, 0])]

In [21]:
from datasets import Dataset, Features, Sequence, Value, DatasetDict

In [22]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [25]:
# from transformers import AutoModelForTokenClassification
# model_name = "Gladiator/microsoft-deberta-v3-large_ner_conll2003"

# tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
# sep_token = tokenizer.sep_token
# sep_token_id = tokenizer.sep_token_id

# model = AutoModelForTokenClassification.from_pretrained(model_name, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)

Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at Gladiator/microsoft-deberta-v3-large_ner_conll2003 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([5]) in the model instantiated
- classifier.weight: found shape torch.Size([9, 1024]) in the checkpoint and torch.Size([5, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [125]:
from transformers import BertForTokenClassification, BertTokenizer, BertForMaskedLM, AutoModelForTokenClassification
from torch import nn

model_name = "microsoft/deberta-base"
# Load pre-trained BERT model for masked language modeling
mask_model = AutoModelForTokenClassification.from_pretrained(model_name,
                                                            num_labels=num_labels,
                                                            id2label=id2label,
                                                            label2id=label2id,
                                                            ignore_mismatched_sizes=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
sep_token = tokenizer.sep_token
sep_token_id = tokenizer.sep_token_id

# Add a new classification layer for NER
# num_labels = 5 # Number of entity labels in your dataset
# model = BertForTokenClassification.from_pretrained(
#     "bert-base-uncased",
    # num_labels=num_labels,
    # id2label=id2label,
    # label2id=label2id,
    # ignore_mismatched_sizes=True
# )

Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForTokenClassification: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'deberta.embeddings.position_embeddings.weight']
- This IS expected if you are initializing DebertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initial

In [61]:
from torch import nn
def set_dropout_p(model, p):
    """
    Recursively set the `p` parameter for all dropout layers in the given PyTorch model.
    """
    for module_name, module in model.named_children():
        if isinstance(module, nn.Dropout):
            module.p = p
        else:
            set_dropout_p(module, p)


In [62]:
set_dropout_p(model, 0.4)
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(0, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.4, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.4, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, el

In [126]:
inputs = tokenizer(names[0], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'Ġpet',
 'max',
 'ĠÐ',
 '±',
 'Ð°',
 'Ð½',
 'ÑĤ',
 'Ð¸',
 'Ðº',
 'ĠÐ',
 '»',
 'Ðµ',
 'Ð¾Ð',
 '¿',
 'Ð°',
 'ÑĢ',
 'Ð´',
 'ĠÐ',
 'º',
 'ÑĢ',
 'Ð°',
 'Ñģ',
 'Ð½',
 'Ġ',
 'ÑĢ',
 'Ð¾Ð',
 '·',
 'Ð¾Ð',
 '¹',
 'Ġ',
 'Ñ',
 'Ī',
 'ÑĤ',
 '[SEP]']

In [ ]:
inputs.word_ids()

In [ ]:
model.config.id2label

In [ ]:
preprocessed_dataset = pd.DataFrame(
    {
        "tokens": [x[0] for x in name_and_labels],
        "labels": [x[1] for x in name_and_labels],
        "numeric_labels": [x[2] for x in name_and_labels],
    }
)

In [ ]:
preprocessed_dataset.head()

In [ ]:
n = len(preprocessed_dataset)
train = preprocessed_dataset[:int(n * 0.8)]
val = preprocessed_dataset[int(n * 0.8):]
dataset_dict = {"train": train, "validation": val}

train_dataset = Dataset.from_dict(dataset_dict["train"])
validation_dataset = Dataset.from_dict(dataset_dict["validation"])

In [124]:
train_dataset["tokens"][:3]

[['petmax', 'бантик', 'леопард', 'красн', 'розой', 'шт'],
 ['бусы', 'для', 'елки', 'шарики_1'],
 ['футболка', 'piazza', 'italia', 'wr']]

In [127]:
inputs = tokenizer(train_dataset[0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'Ġpet',
 'max',
 'ĠÐ',
 '±',
 'Ð°',
 'Ð½',
 'ÑĤ',
 'Ð¸',
 'Ðº',
 'ĠÐ',
 '»',
 'Ðµ',
 'Ð¾Ð',
 '¿',
 'Ð°',
 'ÑĢ',
 'Ð´',
 'ĠÐ',
 'º',
 'ÑĢ',
 'Ð°',
 'Ñģ',
 'Ð½',
 'Ġ',
 'ÑĢ',
 'Ð¾Ð',
 '·',
 'Ð¾Ð',
 '¹',
 'Ġ',
 'Ñ',
 'Ī',
 'ÑĤ',
 '[SEP]']

In [128]:
inputs.word_ids()

[None,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 None]

In [129]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            if label % 2 == 1:
                label += 1
            # here shoud've been changing B-.. to I-... but i've already done it
            new_labels.append(label)

    return new_labels

In [130]:
labels = train_dataset[0]["numeric_labels"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[1, 3, 0, 0, 0, 0]
[-100, 1, 2, 3, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]


In [131]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["numeric_labels"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [132]:
tokenized_train = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=train_dataset.column_names,
)


Map: 100%|████████████████████████████████████████████████████████████████████▉| 17000/17012 [00:01<00:00, 9968.47 examples/s]
                                                                                                                              

In [133]:
tokenized_val = validation_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=train_dataset.column_names,
)


Map:  94%|█████████████████████████████████████████████████████████████████▊    | 4000/4254 [00:00<00:00, 12079.22 examples/s]
                                                                                                                              

In [134]:
tokenized_train[0]

{'labels': [-100,
  1,
  2,
  3,
  4,
  4,
  4,
  4,
  4,
  4,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  -100],
 'input_ids': [1,
  4716,
  29459,
  18697,
  15389,
  26161,
  36765,
  35555,
  35328,
  41171,
  18697,
  2023,
  25482,
  41613,
  9470,
  26161,
  24269,
  37947,
  18697,
  3070,
  24269,
  26161,
  36709,
  36765,
  1437,
  24269,
  41613,
  18400,
  41613,
  9253,
  1437,
  22063,
  23133,
  35555,
  2],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

Padding

In [135]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [136]:
batch = data_collator([tokenized_train[i] for i in range(2)])
batch["labels"]

You're using a DebertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100,    1,    2,    3,    4,    4,    4,    4,    4,    4,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0, -100],
        [-100,    3,    4,    4,    4,    4,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]])

# Metrics

In [137]:
class F1Score:
    def __init__(self):
        self.tp = 0
        self.fp = 0
        self.fn = 0

    def update(self, pred, target):
        pred = frozenset(x for x in pred)
        target = frozenset(x for x in target)
        self.tp += len(pred & target)
        self.fp += len(pred - target)
        self.fn += len(target - pred)

    def reset(self):
        self.tp = 0
        self.fp = 0
        self.fn = 0

    def get(self):
        if self.tp == 0:
            return 0.0
        precision = self.tp / (self.tp + self.fp)
        recall = self.tp / (self.tp + self.fn)
        return 2 / (1 / precision + 1 / recall)


import evaluate

metric = evaluate.load("seqeval")


# custom loss function

In [138]:
# class_weights = [1, 2, 2, 1, 1] # it would be better(maybe) to do 6 and 1 or to not vanish gradient 2/3 and 1 / 3
# class_weights=torch.tensor(class_weights,dtype=torch.float)
criterion = nn.CrossEntropyLoss(reduction='mean')

# Training

In [139]:
# from transformers import TrainingArguments

# args = TrainingArguments(
#     model_name,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=2e-5,
#     num_train_epochs=3,
#     weight_decay=0.01
# )

In [140]:
validation_dataset[:2]

{'tokens': [['лопатка', 'малая', 'бук', 'шт'],
  ['провод', 'пвс', 'гост', 'белый', 'пог']],
 'labels': [['B-GOOD', 'O', 'O', 'O'], ['B-GOOD', 'O', 'O', 'O', 'O']],
 'numeric_labels': [[3, 0, 0, 0], [3, 0, 0, 0, 0]]}

In [141]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_train,
    shuffle=True,
    collate_fn=data_collator,
    batch_size=16,
)
eval_dataloader = DataLoader(
    tokenized_val, collate_fn=data_collator, batch_size=16
)

In [142]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [143]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [144]:
from transformers import get_scheduler

num_train_epochs = 6
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [145]:
output_dir = "../models"

In [146]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

In [147]:
# creating another torch.tensor with labels is a bad idea since seq len can be different for each example pepesad
# so i have to put write labels in dataloader
# also u have to do something with -100 (don't count them, but how?)
def convert_batch_labels(labels: torch.tensor) -> torch.tensor:
    new_labels = torch.zeros(list(labels.shape) + [5])
    for idx, sentence in enumerate(labels):
        new_sentence = torch.zeros(list(sentence.shape) + [5])
        for idx, x in enumerate(sentence):
            if x != -100:
                new_sentence[idx][x] = 1
        new_labels[i] = new_sentence
    return new_labels

def get_labels(dataloader):
    labels = []
    for x in dataloader:
        labels.append(convert_batch_labels(x["labels"]))
    return labels

In [148]:
labels = get_labels(train_dataloader)
labels[0] #where [0 for i in range(5)] - means that it's 100

tensor([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 1.],
         ...,
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        ...,

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         

In [149]:
labels[1][0][9]

tensor([0., 0., 1., 0., 0.])

In [150]:
from tqdm.auto import tqdm

torch.manual_seed(1337)

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for idx, batch in enumerate(train_dataloader):
        outputs = model(**batch)

        #loss = criterion(outputs.logits.flatten(0, 1), labels[idx].flatten(0, 1))
        loss = outputs.loss()
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        break


    model.eval()
    for batch in eval_dataloader:
        break
        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)
        break

    results = metric.compute()
    f1_brand = results["BRAND"]["f1"]
    f1_good = results["GOOD"]["f1"]
    weighted_f1 = 2 / 3 * f1_brand + 1 / 3 * f1_good
    print(f"epoch № {epoch + 1}")
    print(f"WEIGHTED_F1 = {weighted_f1}")
    print(f"f1 brand = {f1_brand}")
    print(f"f1 good = {f1_good}")
    print()
    print(
        f"epoch {epoch}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )

    # Save
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)



  0%|                                                                                                | 0/6384 [15:33<?, ?it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:11                                                                                   │
│                                                                                                  │
│    8 │   # Training                                                                              │
│    9 │   model.train()                                                                           │
│   10 │   for idx, batch in enumerate(train_dataloader):                                          │
│ ❱ 11 │   │   outputs = model(**batch)                                                            │
│   12 │   │                                                                                       │
│   13 │   │   #loss = criterion(outputs.logits.flatten(0, 1), labels[idx].flatten(0, 1))          │
│   14 │   │   loss = outputs.loss()                                                               │
│                                                                                                  │
│ /home/qklent/programming/machine_learning/alfa_bank_receipts/.venv/lib/python3.11/site-packages/ │
│ torch/nn/modules/module.py:1501 in _call_impl                                                    │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ /home/qklent/programming/machine_learning/alfa_bank_receipts/.venv/lib/python3.11/site-packages/ │
│ transformers/models/bert/modeling_bert.py:1758 in forward                                        │
│                                                                                                  │
│   1755 │   │   """                                                                               │
│   1756 │   │   return_dict = return_dict if return_dict is not None else self.config.use_return  │
│   1757 │   │                                                                                     │
│ ❱ 1758 │   │   outputs = self.bert(                                                              │
│   1759 │   │   │   input_ids,                                                                    │
│   1760 │   │   │   attention_mask=attention_mask,                                                │
│   1761 │   │   │   token_type_ids=token_type_ids,                                                │
│                                                                                                  │
│ /home/qklent/programming/machine_learning/alfa_bank_receipts/.venv/lib/python3.11/site-packages/ │
│ torch/nn/modules/module.py:1501 in _call_impl                                                    │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)    

# Evaluating

In [111]:
test_dataset = list(map(lambda x: [' '.join(x[0])] + [x[i] for i in range(1, len(x))], name_and_labels[int(n * 0.8):]))
test_dataset[:3]

[['лопатка малая бук шт', ['B-GOOD', 'O', 'O', 'O'], [3, 0, 0, 0]],
 ['провод пвс гост белый пог',
  ['B-GOOD', 'O', 'O', 'O', 'O'],
  [3, 0, 0, 0, 0]],
 ['зефир асоорти кг денисовы пекарни',
  ['B-GOOD', 'O', 'O', 'B-BRAND', 'I-BRAND'],
  [3, 0, 0, 1, 2]]]

In [112]:
from transformers import pipeline
token_classifier = pipeline(
    "token-classification", model="models", aggregation_strategy="simple"
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/qklent/programming/machine_learning/alfa_bank_receipts/.venv/lib/python3.11/site-packages/ │
│ huggingface_hub/utils/_errors.py:259 in hf_raise_for_status                                      │
│                                                                                                  │
│   256 │   </Tip>                                                                                 │
│   257 │   """                                                                                    │
│   258 │   try:                                                                                   │
│ ❱ 259 │   │   response.raise_for_status()                                                        │
│   260 │   except HTTPError as e:                                                                 │
│   261 │   │   error_code = response.headers.get("X-Error-Code")                                  │
│   262                                                                                            │
│                                                                                                  │
│ /home/qklent/programming/machine_learning/alfa_bank_receipts/.venv/lib/python3.11/site-packages/ │
│ requests/models.py:1021 in raise_for_status                                                      │
│                                                                                                  │
│   1018 │   │   │   )                                                                             │
│   1019 │   │                                                                                     │
│   1020 │   │   if http_error_msg:                                                                │
│ ❱ 1021 │   │   │   raise HTTPError(http_error_msg, response=self)                                │
│   1022 │                                                                                         │
│   1023 │   def close(self):                                                                      │
│   1024 │   │   """Releases the connection back to the pool. Once this method has been            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
HTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/models/resolve/main/config.json

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/qklent/programming/machine_learning/alfa_bank_receipts/.venv/lib/python3.11/site-packages/ │
│ transformers/utils/hub.py:417 in cached_file                                                     │
│                                                                                                  │
│    414 │   user_agent = http_user_agent(user_agent)                                              │
│    415 │   try:                                                                                  │
│    416 │   │   # Load from URL or cache if already cached                                        │
│ ❱  417 │   │   resolved_file = hf_hub_download(                                                  │
│    418 │   │   │   path_or_repo_id,                                                              │
│    419 │   │   │   filename,                                                                     │
│    420 │   │   │   subfolder=None if len(subfolder) == 0 else subfolder,                         │
│                                                                                                  │
│ /home/qklent/programming/machine_learning/alfa_bank_receipts/.venv/lib/python3.11/site-packages/ │
│ huggingface_hub/utils/_validators.py:118 in _inner_fn                                            │
│                                                                                     

In [ ]:
out_brands = []
out_goods = []
for name, label, ner_tags in test_dataset:
    curr_b = []
    curr_g = []
    out = token_classifier(name)
    for x in out:
        if x["entity_group"] == "BRAND":
            curr_b.append(x["word"])
        else:
            curr_g.append(x["word"])
    out_brands.append(curr_b)
    out_goods.append(curr_g)

In [ ]:
out_goods = list(map(lambda x: ','.join(x), out_goods))
out_goods[:10]

In [ ]:
out_brands = list(map(lambda x: ','.join(x), out_brands))
out_brands[:10]

In [ ]:
class F1Score:
    def __init__(self):
        self.tp = 0
        self.fp = 0
        self.fn = 0

    def update(self, pred, target):
        pred = frozenset(x for x in pred)
        target = frozenset(x for x in target)
        self.tp += len(pred & target)
        self.fp += len(pred - target)
        self.fn += len(target - pred)

    def reset(self):
        self.tp = 0
        self.fp = 0
        self.fn = 0

    def get(self):
        if self.tp == 0:
            return 0.0
        precision = self.tp / (self.tp + self.fp)
        recall = self.tp / (self.tp + self.fn)
        return 2 / (1 / precision + 1 / recall)


In [ ]:
f1_brand = F1Score()
f1_good = F1Score()
f1_brand.update(out_brands, real_brands)
f1_good.update(out_goods, real_goods)

f1 = 2 / 3 * f1_brand.get() + 1 / 3 * f1_good.get()
f1, f1_brand.get(), f1_good.get()

In [ ]:
brands = []
goods = []
for x in test_dataset.name:
    curr_b = []
    curr_g = []
    out = token_classifier(x)
    for x in out:
        if x["entity_group"] == "BRAND":
            curr_b.append(x["word"])
        else:
            curr_g.append(x["word"])
    brands.append(curr_b)
    goods.append(curr_g)

In [ ]:
subm = pd.DataFrame(
    {
        "id": [i for i in range(len(brands))],
        "good": goods,
        "brand": brands
    }
)
subm.head()

In [ ]:
subm.good = subm.good.apply(lambda x: ','.join(x))
subm.brand = subm.brand.apply(lambda x: ','.join(x))
subm.head()

In [ ]:
subm.to_csv("yoyo.csv")

In [ ]:
test[:10]

,tokens,labels,numeric_labels
17012,"[лопатка, малая, бук, шт]","[B-GOOD, O, O, O]","[3, 0, 0, 0]"
17013,"[провод, пвс, гост, белый, пог]","[B-GOOD, O, O, O, O]","[3, 0, 0, 0, 0]"
17014,"[зефир, асоорти, кг, денисовы, пекарни]","[B-GOOD, O, O, B-BRAND, I-BRAND]","[3, 0, 0, 1, 2]"
17015,"[шампунь, чистая, линия, сила, гус]","[B-GOOD, B-BRAND, I-BRAND, O, O]","[3, 1, 2, 0, 0]"
17016,"[сыворотка, для, секущихся, кончиков, мл, kera...","[B-GOOD, O, O, O, O, O, O, O, O]","[3, 0, 0, 0, 0, 0, 0, 0, 0]"
17017,"[бестабачная, смесь, brusko, гр, яблоко, мятой...","[B-GOOD, I-GOOD, B-BRAND, O, O, O, O, O]","[3, 4, 1, 0, 0, 0, 0, 0]"
17018,"[пена, монтажная, tytan, мл, std, стандарт]","[B-GOOD, O, B-BRAND, O, O, O]","[3, 0, 1, 0, 0, 0]"
17019,"[печень, трески, гр, ст, штурвал]","[B-GOOD, O, O, O, B-BRAND]","[3, 0, 0, 0, 1]"
17020,"[yh, трусы, боксеры, мужские]","[O, B-GOOD, O, O]","[0, 3, 0, 0]"
17021,"[длинногубцы, тонкогубцы, smartbuy, мм]","[B-GOOD, O, B-BRAND, O]","[3, 0, 1, 0]"
